# etf-analysis-ver1.ipynb

SBI 証券で購入可能な ETF を対象に分析


In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import datetime
from dateutil import relativedelta
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import sqlite3
from pathlib import Path
from etf_dot_com import get_etf_fundamentals, get_tickers_from_db, save_data_to_db
from tqdm import tqdm
import yfinance as yf
# from curl_cffi import requests
# from curl_cffi.requests import Session
import curl_cffi

ROOT_DIR = Path().cwd().parent
ETF_DIR = ROOT_DIR / "data/ETF"


## 日次データ download


### 国内 ETF


In [ ]:
etf_list_JP = [
    s + ".T"
    for s in pd.read_csv(ETF_DIR / "SBI証券_購入可能ETF_国内.csv")[
        "ティッカー"
    ].tolist()
]
session = curl_cffi.Session(impersonate="safari15_5")
data = yf.download(etf_list_JP, session=session, period="max", interval="1d")
display(data)


C:\Users\Yuki Hata\AppData\Local\Temp\ipykernel_3708\3132687855.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(etf_list_JP, session=session, period="max", interval="1d")
[*********************100%***********************]  398 of 398 completed

2 Failed downloads:
['1553.T']: YFTzMissingError('possibly delisted; no timezone found')
['345A.T']: YFInvalidPeriodError("345A.T: Period 'max' is invalid, must be one of: 1d, 5d")


Price      Adj Close               Close                                     \
Ticker        1553.T 345A.T       1305.T       1306.T       1308.T   1309.T   
Date                                                                          
2008-01-04       NaN    NaN  1129.811035  1111.187622  1109.043091      NaN   
2008-01-07       NaN    NaN  1114.200806  1095.945190  1093.736450      NaN   
2008-01-08       NaN    NaN  1122.672607  1104.137207  1101.980591      NaN   
2008-01-09       NaN    NaN  1139.537842  1120.676270  1118.600220      NaN   
2008-01-10       NaN    NaN  1121.182129  1102.708374  1100.558350      NaN   
...              ...    ...          ...          ...          ...      ...   
2025-09-22       NaN    NaN  3322.000000  3286.000000  3248.000000  48960.0   
2025-09-24       NaN    NaN  3327.000000  3293.000000  3255.000000  52660.0   
2025-09-25       NaN    NaN  3343.000000  3312.000000  3269.000000  51510.0   
2025-09-26       NaN    NaN  3345.000000  3306.000000  3270.000000  52600.0   
2025-09-29       NaN    NaN  3325.000000  3290.000000  3252.000000  52900.0   

Price                                                  ...   Volume         \
Ticker           1311.T      1319.T   1320.T   1321.T  ...   381A.T 382A.T   
Date                                                   ...                   
2008-01-04   663.452820  209.437790      NaN      NaN  ...      NaN    NaN   
2008-01-07   654.499695  206.485931      NaN      NaN  ...      NaN    NaN   
2008-01-08   662.574524  208.361740      NaN      NaN  ...      NaN    NaN   
2008-01-09   672.723267  211.546295      NaN      NaN  ...      NaN    NaN   
2008-01-10   660.662842  207.852814      NaN      NaN  ...      NaN    NaN   
...                 ...         ...      ...      ...  ...      ...    ...   
2025-09-22  1644.500000  605.000000  46730.0  46910.0  ...     15.0  209.0   
2025-09-24  1648.000000  605.000000  46900.0  47070.0  ...      2.0  457.0   
2025-09-25  1670.000000  606.299988  46990.0  47180.0  ...      2.0    8.0   
2025-09-26  1667.000000  608.099976  46640.0  46790.0  ...  36458.0  254.0   
2025-09-29  1650.000000  598.000000  46560.0  46710.0  ...   3005.0    1.0   

Price                                                                       \
Ticker       383A.T  394A.T   395A.T  396A.T   399A.T    404A.T     408A.T   
Date                                                                         
2008-01-04      NaN     NaN      NaN     NaN      NaN       NaN        NaN   
2008-01-07      NaN     NaN      NaN     NaN      NaN       NaN        NaN   
2008-01-08      NaN     NaN      NaN     NaN      NaN       NaN        NaN   
2008-01-09      NaN     NaN      NaN     NaN      NaN       NaN        NaN   
2008-01-10      NaN     NaN      NaN     NaN      NaN       NaN        NaN   
...             ...     ...      ...     ...      ...       ...        ...   
2025-09-22   7906.0  2290.0   7030.0  4520.0  51032.0   38955.0   883280.0   
2025-09-24    377.0  1570.0   3030.0   220.0  88433.0  247652.0  1016270.0   
2025-09-25  55009.0  2290.0  22740.0   940.0  20074.0  360367.0   597310.0   
2025-09-26   1299.0   600.0  39400.0  1420.0  92339.0  188460.0   560760.0   
2025-09-29   3110.0   100.0  35660.0   150.0  96220.0  263823.0   620990.0   

Price                
Ticker       413A.T  
Date                 
2008-01-04      NaN  
2008-01-07      NaN  
2008-01-08      NaN  
2008-01-09      NaN  
2008-01-10      NaN  
...             ...  
2025-09-22  23288.0  
2025-09-24  16462.0  
2025-09-25  16328.0  
2025-09-26  23808.0  
2025-09-29   4121.0  

[4359 rows x 1992 columns]

In [ ]:
df = data.stack(future_stack=True).reset_index()
df = pd.melt(df, id_vars=["Date", "Ticker"], value_vars=df.columns[2:])
display(df)


,Date,Ticker,Price,value
0,2008-01-04,1553.T,Adj Close,NaN
1,2008-01-04,345A.T,Adj Close,NaN
2,2008-01-04,1305.T,Adj Close,NaN
3,2008-01-04,1306.T,Adj Close,NaN
4,2008-01-04,1308.T,Adj Close,NaN
...,...,...,...,...
10409287,2025-09-29,396A.T,Volume,150.0
10409288,2025-09-29,399A.T,Volume,96220.0
10409289,2025-09-29,404A.T,Volume,263823.0
10409290,2025-09-29,408A.T,Volume,620990.0


In [ ]:
des = yf.Ticker("1306.T").holdings
display(des)


AttributeError: 'Ticker' object has no attribute 'holdings'

## 海外 ETF


In [ ]:
df_etf_list = pd.read_csv(ETF_DIR / "SBI証券_購入可能ETF.csv")[
    ["Category", "Asset Type", "ティッカー"]
].rename(columns={"ティッカー": "Ticker"})
ticker_list = ",".join(df_etf_list["Ticker"].unique().tolist() + ["^SPX"])

session = curl_cffi.Session(impersonate="safari15_5")
Tickers_obj = yf.Tickers(ticker_list, session=session)
data = (
    Tickers_obj.history(period="max", interval="1d")
    .stack(future_stack=True)
    .reset_index()
    .rename(columns={"Date": "date"})
)
display(data)

df = pd.melt(
    data,
    id_vars=["date", "Ticker"],
    value_vars=data.columns.tolist()[2:],
    var_name="variable",
    value_name="value",
).assign(date=lambda row: pd.to_datetime(row["date"], unit="s", utc=True))
display(df)

df = (
    pd.merge(
        df[df["variable"] == "Close"]
        .drop(columns=["variable"])
        .rename(columns={"value": "Close"}),
        df_etf_list,
        on=["Ticker"],
        how="left",
    )
    .sort_values("date")
    .assign(
        year=lambda row: pd.to_datetime(row["date"]).dt.year,
        month=lambda row: pd.to_datetime(row["date"]).dt.month,
        date=lambda row: pd.to_datetime(row["date"]).dt.strftime("%Y-%m-%d"),
    )
)

# calculate log return
df["log_price"] = np.log(df["Close"])
df["log_return"] = df.groupby(["Ticker"])["log_price"].diff()
display(df)
start_date = "2022-01-01"

# calculate correlation
log_return_wide = df[df["date"] >= start_date].pivot(
    index="date", columns="Ticker", values="log_return"
)
corr_matrix = log_return_wide.corr().reset_index().rename(columns={"Ticker": "Ticker1"})
df_corr = pd.melt(
    corr_matrix,
    id_vars="Ticker1",
    value_vars=corr_matrix.columns.tolist()[1:],
    var_name="Ticker2",
    value_name="correlation",
)
# 自己相関と重複ペアを削除
df_corr = df_corr[df_corr["Ticker1"] < df_corr["Ticker2"]].copy()
df_corr = df_corr.sort_values("correlation", ignore_index=True)
display(df_corr)


[*******************   39%                       ]  29 of 74 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: CSOP"}}}
[********************  42%                       ]  31 of 74 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: UOBA"}}}
[*********************100%***********************]  74 of 74 completed

2 Failed downloads:
['UOBA', 'CSOP']: YFTzMissingError('possibly delisted; no timezone found')


Price,date,Ticker,Adj Close,Capital Gains,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,1927-12-30,CSOP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1927-12-30,UOBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1927-12-30,BBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1927-12-30,COM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1927-12-30,CURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1816769,2025-09-26,XLRE,NaN,0.0,41.880001,0.0,41.966000,41.514999,41.610001,0.0,3.436200e+06
1816770,2025-09-26,XLU,NaN,0.0,86.809998,0.0,86.839996,85.629997,85.709999,0.0,8.116900e+06
1816771,2025-09-26,XLV,NaN,0.0,135.500000,0.0,135.619995,134.639999,135.149994,0.0,1.007050e+07
1816772,2025-09-26,XLY,NaN,0.0,239.710007,0.0,239.850006,236.470001,237.289993,0.0,4.874700e+06


,date,Ticker,variable,value
0,1927-12-30 00:00:00+00:00,CSOP,Adj Close,NaN
1,1927-12-30 00:00:00+00:00,UOBA,Adj Close,NaN
2,1927-12-30 00:00:00+00:00,BBH,Adj Close,NaN
3,1927-12-30 00:00:00+00:00,COM,Adj Close,NaN
4,1927-12-30 00:00:00+00:00,CURE,Adj Close,NaN
...,...,...,...,...
16350961,2025-09-26 00:00:00+00:00,XLRE,Volume,3.436200e+06
16350962,2025-09-26 00:00:00+00:00,XLU,Volume,8.116900e+06
16350963,2025-09-26 00:00:00+00:00,XLV,Volume,1.007050e+07
16350964,2025-09-26 00:00:00+00:00,XLY,Volume,4.874700e+06


,date,Ticker,Close,Category,Asset Type,year,month,log_price,log_return
0,1927-12-30,CSOP,NaN,Commodity,Real Estate,1927,12,NaN,NaN
53,1927-12-30,TECS,NaN,US,Technology,1927,12,NaN,NaN
52,1927-12-30,TECL,NaN,US,Technology,1927,12,NaN,NaN
51,1927-12-30,TAN,NaN,US,Clean Energy,1927,12,NaN,NaN
50,1927-12-30,SUSA,NaN,US,ESG,1927,12,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1816721,2025-09-26,IAU,70.989998,Commodity,Gold,2025,9,4.262539,0.005792
1816720,2025-09-26,GLDM,74.589996,Commodity,Gold,2025,9,4.312006,0.005647
1816719,2025-09-26,GLD,346.739990,Commodity,Gold,2025,9,5.848575,0.005756
1816716,2025-09-26,FXN,16.799999,US,Energy,2025,9,2.821379,0.010171


,Ticker1,Ticker2,correlation
0,ERY,XLE,-0.997045
1,ERX,ERY,-0.996421
2,DUST,GDX,-0.996321
3,DRV,XLRE,-0.995695
4,FAZ,XLF,-0.995212
...,...,...,...
2696,UOBA,XLV,NaN
2697,CSOP,XLY,NaN
2698,UOBA,XLY,NaN
2699,CSOP,^SPX,NaN


### Calculate Rolling Correlation


In [ ]:
start_date = "2022-01-01"

# calculate correlation
rolling_period = 365  # unit: days
df["date"] = pd.to_datetime(df["date"])
all_date = sorted(df["date"].unique().tolist(), reverse=True)
date_range = [
    d
    for d in all_date
    if (pd.to_datetime(d) <= datetime.datetime.today())
    and (
        pd.to_datetime(d)
        > (datetime.datetime.today() - datetime.timedelta(days=365 * 10))
    )
]


dfs_corr = []
for end_date in tqdm(date_range):
    start_date = end_date - datetime.timedelta(days=365)
    log_return_wide = df.loc[
        (df["date"] >= start_date) & (df["date"] < end_date)
    ].pivot(index="date", columns="Ticker", values="log_return")
    corr_matrix = (
        log_return_wide.corr().reset_index().rename(columns={"Ticker": "Ticker1"})
    )
    df_corr = pd.melt(
        corr_matrix,
        id_vars="Ticker1",
        value_vars=corr_matrix.columns.tolist()[1:],
        var_name="Ticker2",
        value_name="correlation",
    )
    # 自己相関と重複ペアを削除
    df_corr = df_corr[df_corr["Ticker1"] < df_corr["Ticker2"]].copy()
    df_corr = df_corr.sort_values("correlation", ignore_index=True).assign(
        date=end_date
    )

    dfs_corr.append(df_corr)

df_corr = pd.concat(dfs_corr, ignore_index=True)
display(df_corr)


100%|██████████| 2510/2510 [01:02<00:00, 40.19it/s]


,Ticker1,Ticker2,correlation,date
0,DUST,GDX,-0.996255,2025-09-26
1,DRV,XLRE,-0.995839,2025-09-26
2,ERY,XLE,-0.995749,2025-09-26
3,DUST,NUGT,-0.995061,2025-09-26
4,LABD,LABU,-0.994683,2025-09-26
...,...,...,...,...
6779505,SPYD,^SPX,NaN,2015-10-05
6779506,SROI,^SPX,NaN,2015-10-05
6779507,UOBA,^SPX,NaN,2015-10-05
6779508,XLC,^SPX,NaN,2015-10-05


### vs SPX で correlation の時系列が安定している ETF を抽出


In [ ]:
df_corr_spx = df_corr[df_corr["Ticker2"] == "^SPX"]
# display(df_corr_spx)
g = (
    df_corr_spx.groupby("Ticker1")["correlation"]
    .agg(["mean", "std"])
    .sort_values("mean", key=lambda x: x.abs())
)
display(g.head(20))


,mean,std
Ticker1,,
IAU,0.009702,0.183987
GLD,0.010434,0.184028
GLDM,0.082302,0.165466
DUST,-0.169806,0.172877
NUGT,0.174569,0.174271
GDX,0.176908,0.174414
IGLD,0.178305,0.152894
COM,0.181450,0.111894
SLV,0.182512,0.135492


In [ ]:
df_plot = df_corr_spx.copy()
display(df_plot)
df_plot = pd.pivot(df_plot, index="date", columns="Ticker1", values="correlation")
display(df_plot)
# df_plot = df_plot.set_index("date")
# df_plot.index = pd.to_datetime(df_plot.index)

fig = go.Figure()
for ticker in [
    "IAU",
    "GLD",
    "GLDM",
    "DUST",
    "NUGT",
    "GDX",
    "IGLD",
    "COM",
    "SLV",
    "GDXJ",
]:

    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot[ticker],
            mode="lines",
            name=ticker,
            line=dict(width=0.8),
        )
    )
fig.update_layout(
    title="Rolling Correlation: Window=365 days",
    xaxis_title="Date",
    yaxis_title="Correlation",
    yaxis=dict(range=[-1, 1]),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    margin=dict(l=20, r=20, t=30, b=20),
    template="plotly_dark",
)


,Ticker1,Ticker2,correlation,date
19,TECS,^SPX,-0.934298,2025-09-26
31,FAZ,^SPX,-0.848846,2025-09-26
89,LABD,^SPX,-0.661900,2025-09-26
124,ERY,^SPX,-0.608342,2025-09-26
144,DRV,^SPX,-0.597609,2025-09-26
...,...,...,...,...
6779505,SPYD,^SPX,NaN,2015-10-05
6779506,SROI,^SPX,NaN,2015-10-05
6779507,UOBA,^SPX,NaN,2015-10-05
6779508,XLC,^SPX,NaN,2015-10-05


Ticker1,BBH,COM,CSOP,CURE,DPST,DRN,DRV,DUST,DVY,ERX,...,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
date,,,,,,,,,,,,,,,,,,,,,
2015-10-05,0.706335,NaN,NaN,0.872480,0.621236,0.635702,-0.637954,-0.178910,0.913683,0.717491,...,NaN,0.718384,0.927162,0.945065,0.935382,0.865339,NaN,0.578778,0.876771,0.925249
2015-10-06,0.698630,NaN,NaN,0.868318,0.631878,0.639882,-0.642273,-0.188584,0.914837,0.721457,...,NaN,0.722406,0.928207,0.944894,0.936195,0.867311,NaN,0.581841,0.872119,0.925576
2015-10-07,0.695562,NaN,NaN,0.863385,0.632018,0.640294,-0.642634,-0.186911,0.914829,0.716608,...,NaN,0.717310,0.928258,0.944774,0.935836,0.867122,NaN,0.582173,0.867530,0.925648
2015-10-08,0.694673,NaN,NaN,0.862332,0.630361,0.640343,-0.642741,-0.180610,0.914637,0.717067,...,NaN,0.717702,0.927685,0.945011,0.935033,0.868003,NaN,0.582124,0.866499,0.924445
2015-10-09,0.689807,NaN,NaN,0.859890,0.631863,0.636119,-0.638456,-0.162761,0.913587,0.718554,...,NaN,0.719289,0.926710,0.944290,0.933889,0.866680,NaN,0.577734,0.864022,0.924039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-22,0.592963,0.276801,NaN,0.573782,0.713694,0.586966,-0.599415,-0.186764,0.778727,0.614567,...,0.889876,0.616467,0.845012,0.900509,0.942300,0.394603,0.588685,0.460603,0.577450,0.893110
2025-09-23,0.592780,0.277447,NaN,0.573757,0.711987,0.587134,-0.599543,-0.187709,0.778220,0.614373,...,0.888897,0.616228,0.844578,0.900522,0.941918,0.391491,0.588846,0.460929,0.577351,0.892450
2025-09-24,0.595105,0.274992,NaN,0.573041,0.712298,0.585266,-0.597655,-0.188143,0.775894,0.610752,...,0.888797,0.612434,0.844905,0.900237,0.942109,0.389617,0.586886,0.459649,0.576595,0.893046


In [ ]:
df_cum_return = np.exp(log_return_wide.cumsum())
first_value = df_cum_return.iloc[0]
df_cum_return = df_cum_return.div(first_value)
df_cum_return.index = pd.to_datetime(df_cum_return.index)

# display(df_cum_return)
# display(df_corr[df_corr["Ticker2"] == "^SPX"])

cum_return_latest = pd.melt(
    df_cum_return.reset_index(),
    id_vars="date",
    value_vars=df_cum_return.columns,
    var_name="Ticker",
).set_index("date")

cum_return_latest = (
    cum_return_latest.loc[df_cum_return.index[-1]]
    .sort_values("value")
    .dropna()
    .reset_index(drop=True)
)

benchmark_beat_performers = cum_return_latest.loc[
    cum_return_latest[cum_return_latest["Ticker"] == "^SPX"].index[0] + 1 :, "Ticker"
].tolist()


# --- plot ---
# fig, ax = plt.subplots(figsize=(10, 6), tight_layout=True)
# for ticker in benchmark_beat_performers + ["^SPX"]:

#     if ticker == "^SPX":
#         sns.lineplot(
#             df_cum_return,
#             x=df_cum_return.index,
#             y=df_cum_return[ticker],
#             ax=ax,
#             label=ticker,
#             linewidth=0.8,
#             color="black",
#         )
#     else:
#         sns.lineplot(
#             df_cum_return,
#             x=df_cum_return.index,
#             y=df_cum_return[ticker],
#             ax=ax,
#             label=ticker,
#             linewidth=0.8,
#         )
# ax.set_ylabel("Cumulative Return")
# ax.grid()
# ax.legend(loc="upper left", ncols=4)
# plt.show()


fig = go.Figure()

for ticker in benchmark_beat_performers + ["^SPX"]:
    if ticker == "^SPX":
        fig.add_trace(
            go.Scatter(
                x=df_cum_return.index,
                y=df_cum_return[ticker],
                name=ticker,
                line=dict(color="white", width=0.8),  # 線を黒色に
            )
        )
    else:
        fig.add_trace(
            go.Scatter(
                x=df_cum_return.index,
                y=df_cum_return[ticker],
                name=ticker,
                line=dict(width=0.8),  # 線の太さ
            )
        )

fig.update_layout(
    width=1200,
    height=600,
    yaxis_title="Cumulative Return",
    margin=dict(l=20, r=20, t=30, b=20),
    # legend=dict(x=0.01, y=0.99, xanchor="left", yanchor="top"),
    template="plotly_dark",
)

fig.show()


In [ ]:
df_etf = pd.read_csv(ETF_DIR / "SBI証券_購入可能ETF.csv").rename(
    columns={"ティッカー": "Ticker"}
)
display(
    df_etf[
        df_etf["Ticker"].isin(
            ["GDXJ", "GDX", "XLK", "SUSA", "TECS", "VDE", "NUGT", "FAS", "DUST"]
        )
    ].sort_values("Ticker")
)


,Category,Asset Type,Ticker,名称,連動指標,市場,ファクトシート,コメント
12,Commodity,Gold,DUST,Direxion デイリー 金鉱株 ベア2倍 ETF,NYSE Arca ゴールド マイナーズ インデックス,NYSE Arca,http://search.sbisec.co.jp/v3/ex/etf_DUST_1605...,--
53,US,Financials,FAS,Direxion デイリー米国金融株ブル3倍 ETF,ラッセル1000金融株指数,NYSE Arca,https://www.direxion.com/uploads/FAS-FAZ-Fact-...,ラッセル1000金融株指数の300%のパフォーマンスに連動した成果を目指す。
13,Commodity,Gold,GDX,ヴァンエック ベクトル 金鉱株ETF,NYSE Arca金鉱株インデックス,NYSE Arca,https://www.vaneck.com/ucits/library/fact-shee...,金鉱株ETFは、NYSE Arca金鉱株インデックスの価格と利回り（手数料と経費控除前）に連...
14,Commodity,Gold,GDXJ,ヴァンエック ベクトル ジュニア金鉱株 ETF,ヴァンエック・ベクトル中小型金鉱株インデックス,NYSE Arca,https://www.vaneck.com/ucits/library/fact-shee...,中小型金鉱株ETFは、ヴァンエック・ベクトル中小型金鉱株インデックスの価格と利回り（手数料と...
19,Commodity,Gold,NUGT,Direxion デイリー 金鉱株 ブル2倍 ETF,NYSE Arca ゴールド マイナーズ インデックス,NYSE Arca,https://www.direxion.com/uploads/NUGT-DUST-Fac...,--
51,US,ESG,SUSA,iシェアーズ ESG オプティマイズド 米国ETF,MSCI USA エクステンデッド ESG セレクト・インデックス,NYSE Arca,https://www.blackrock.com/jp/individual/ja/lit...,MSCI USA ESG セレクト ソーシャル インデックスの価格および利回り実績と同等水準...
70,US,Technology,TECS,Direxion デイリー テクノロジー株 ベア 3倍 ETF,テクノロジー・セレクト・セクター指数,NYSE Arca,https://www.direxion.com/uploads/TECL-TECS-Fac...,テクノロジー・セレクト・セクター指数の日次運用実績の逆数の300%に連動する投資成果を目指し...
46,US,Energy,VDE,バンガード 米国エネルギー・セクターETF,MSCI USインベスタブル・マーケット・エネルギー25/50インデックス,NYSE Arca,https://institutional.vanguard.com/iippdf/pdfs...,米国のエネルギー・セクターの大型株、中型株、小型株を網羅する、MSCI USインベスタブル・...
71,US,Technology,XLK,テクノロジー セレクト セクター SPDR ファンド,S&P テクノロジー セレクト セクター指数,NYSE Arca,https://www.ssga.com/library-content/products/...,テクノロジー・セレクト・セクター SPDRファンドは、S&Pテクノロジー・セレクト・セクター...
